<a href="https://colab.research.google.com/github/gseetha04/LLMs-Medicaldata/blob/main/Gestationaldiabetes_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
import pandas as pd
data_GD = pd.read_excel('/content/final_merged.xlsx')

In [ ]:
data_GD

,Unnamed: 0,Unnamed: 0_x,text,variables_x,role_x,Unnamed: 0_y,variables_y,role_y
0,0,58,"If the screening threshold is met or exceeded,...",If the screening threshold is met or exceeded,condition,44,screening threshold is met or exceeded,condition
1,1,58,"If the screening threshold is met or exceeded,...",If the screening threshold is met or exceeded,condition,45,oral glucose tolerance test (OGTT),action
2,2,59,"If the screening threshold is met or exceeded,...",patients receive the oral glucose tolerance te...,action,44,screening threshold is met or exceeded,condition
3,3,59,"If the screening threshold is met or exceeded,...",patients receive the oral glucose tolerance te...,action,45,oral glucose tolerance test (OGTT),action
4,4,60,"During the OGTT, a fasting glucose level is ob...","a fasting glucose level is obtained, followed ...",action,46,OGTT,condition
...,...,...,...,...,...,...,...,...
100,100,2298,Fetal macrosomia (LGA) is a major indicator of...,Fetal macrosomia (LGA),effect,565,Fetal macrosomia,signal
101,101,2299,Fetal macrosomia (LGA) is a major indicator of...,hyperglycemia during pregnancy,cause,565,Fetal macrosomia,signal
102,102,2328,Measuring FPG alone identified 8.3% of the coh...,Measuring FPG alone,cause,570,Measuring FPG alone,cause
103,103,2329,Measuring FPG alone identified 8.3% of the coh...,identified 8.3% of the cohort as having GDM,effect,570,Measuring FPG alone,cause


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = "distilbert-base-uncased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
# import pandas as pd
# data_arslan = pd.read_excel('/content/data_causal.xlsx',sheet_name='Sheet3')

In [ ]:
import pandas as pd
data_final = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')

In [ ]:
data_arslan

,sentences
0,ImportanceGestational diabetes is diabetes tha...
1,Rationale <CO>Pregnant persons at 24 weeks of ...
2,There is <S>inadequate evidence</S> that <C>c...
3,<S>Benefits</S> of <E>early detection and int...
4,There is <S>inadequate direct evidence</S> th...
...,...
1056,The <C>impact of the interventions</C> was obs...
1057,<C>Lifestyle interventions</C> to prevent prog...
1058,Moreover the AAFP is concerned that <S>harms</...
1059,<A>There is also concern that persons of color...


In [ ]:
data_final

,final
0,<S>Pregnant persons</S> <C>with gestational di...
1,<C>Gestational diabetes</C> has also been ass...
2,EVIDENCE ASSESSMENT The USPSTF concludes with ...
3,RECOMMENDATION The USPSTF recommends <A>screen...
4,IImportance<C>Gestational diabetes</C> is dia...
...,...
285,Clinical Preventive Service RecommendationDiab...
286,The AAFP also agrees that there is insufficien...
287,"While AAFP acknowledges that <E>diabetes</E> ,..."
288,The evidence did show a <E>reduction in the in...


In [ ]:
# data_arslan['text'] = data_arslan['sentences'].replace(',','', regex=True)
# data_arslan['text'] = data_arslan['sentences'].replace(':','', regex=True)
# data_arslan['text'] = data_arslan['sentences'].replace(';','', regex=True)
# data_arslan['text'] = data_arslan['sentences'].replace('(','')
# data_arslan['text'] = data_arslan['sentences'].str.replace(')','')
# data_arslan['text'] = data_arslan['sentences'].str.replace('[','')
# data_arslan['text'] = data_arslan['sentences'].str.replace(']','')

In [ ]:
data_arslan

,Unnamed: 0.1,Unnamed: 0,sentences,text
0,5,5,ImportanceGestational diabetes is diabetes tha...,ImportanceGestational diabetes is diabetes tha...
1,7,7,"Rationale', '<CO>Pregnant persons at 24 weeks ...","Rationale', '<CO>Pregnant persons at 24 weeks ..."
2,8,8,"', 'There is <S>inadequate evidence</S> that <...","', 'There is <S>inadequate evidence</S> that <..."
3,9,9,"', '<S>Benefits</S> of <E>early detection and ...","', '<S>Benefits</S> of <E>early detection and ..."
4,10,10,"', 'There is <S>inadequate direct evidence</S>...","', 'There is <S>inadequate direct evidence</S>..."
...,...,...,...,...
1056,16,16,The <C>impact of the interventions</C> was obs...,The <C>impact of the interventions</C> was obs...
1057,17,17,<C>Lifestyle interventions</C> to prevent prog...,<C>Lifestyle interventions</C> to prevent prog...
1058,18,18,"Moreover, the AAFP is concerned that <S>harms<...","Moreover, the AAFP is concerned that <S>harms<..."
1059,20,20,<A>There is also concern that persons of color...,<A>There is also concern that persons of color...


In [ ]:
import pandas as pd
import spacy
import re
import json
from nltk.tokenize import word_tokenize

data_train_df = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')
tagged_words = []
for para in data_train_df['final']:
   para=para.encode("ascii", "ignore")
   para=para.decode()
   connl_text= []
   label = []
   cause = re.findall(r'<C>(.*?)</C>', para)
   condition = re.findall(r'<CO>(.*?)</CO>', para)
   effect = re.findall(r'<E>(.*?)</E>', para)
   action = re.findall(r'<A>(.*?)</A>', para)
   signal = re.findall(r'<S>(.*?)</S>',para)
   #modal = re.findall(r'<>')
  # ARG0 = [re.sub('<(\w+)>(.*?)<\/\w+>','',i) for i in ARG0]
   splitted = re.split('<C>|</C>|<CO>|</CO>|<E>|</E>|<A>|</A>|<S>|</S>', para)  # splitted considering tags
   if cause or condition or effect or action or signal:  # if any tag in sentence
    for split in splitted:  # search each index
            if split in cause:
                for token in split.split():
                    connl_text.append(token)
                    label.append('C')
            elif split in effect:
                for token in split.split():
                    connl_text.append(token)
                    label.append('E')
            elif split in signal:
                for token in split.split():
                  connl_text.append(token)
                  label.append('S')
            elif split in action:
                for token in split.split():
                  connl_text.append(token)
                  label.append('A')
            elif split in condition:
                for token in split.split():
                  connl_text.append(token)
                  label.append('CO')
            else:  # current word is not an entity
                for token in split.split():
                    connl_text.append(token)
                    label.append('O')

    tagged_words.append({'Word': connl_text, 'Tag': label})

In [ ]:
data_train = pd.DataFrame(tagged_words)

In [ ]:
data_train

,Word,Tag
0,"[Pregnant, persons, with, gestational, diabete...","[S, S, C, C, C, O, O, E, E, E, E, E, E, E, E, ..."
1,"[Gestational, diabetes, has, also, been, assoc...","[C, C, O, O, O, O, O, O, E, E, E, E, E, E, E, ..."
2,"[EVIDENCE, ASSESSMENT, The, USPSTF, concludes,...","[O, O, O, O, O, O, O, O, O, O, O, O, S, S, S, ..."
3,"[RECOMMENDATION, The, USPSTF, recommends, scre...","[O, O, O, O, A, A, A, A, O, CO, CO, CO, CO, CO..."
4,"[IImportance, Gestational, diabetes, is, diabe...","[O, C, C, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
284,"[Clinical, Preventive, Service, Recommendation...","[O, O, O, O, O, O, O, O, O, O, A, A, A, A, A, ..."
285,"[The, AAFP, also, agrees, that, there, is, ins...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
286,"[While, AAFP, acknowledges, that, diabetes, ,,...","[O, O, O, O, E, O, O, O, O, O, O, O, O, O, O, ..."
287,"[The, evidence, did, show, a, reduction, in, t...","[O, O, O, O, O, E, E, E, E, E, E, O, O, O, O, ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_train[["Word", "Tag"]].rename(columns={"Word": "X", "Tag": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

In [ ]:
X_text_new = pd.DataFrame(X_raw,columns=['sentences'])
y_text_new = pd.DataFrame(y_raw,columns=['label'])

In [ ]:
df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

In [ ]:
df_train_new['token_count'] = df_train_new.sentences.str.split().str.len()

In [ ]:
df_train_new['label_count'] = df_train_new.label.str.split().str.len()

In [ ]:
df_train_new

,sentences,label,token_count,label_count
0,"[""Pregnant"", ""persons"", ""with"", ""gestational"",...","[""S"", ""S"", ""C"", ""C"", ""C"", ""O"", ""O"", ""E"", ""E"", ...",30,30
1,"[""Gestational"", ""diabetes"", ""has"", ""also"", ""be...","[""C"", ""C"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""E"", ...",25,25
2,"[""EVIDENCE"", ""ASSESSMENT"", ""The"", ""USPSTF"", ""c...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",34,34
3,"[""RECOMMENDATION"", ""The"", ""USPSTF"", ""recommend...","[""O"", ""O"", ""O"", ""O"", ""A"", ""A"", ""A"", ""A"", ""O"", ...",20,20
4,"[""IImportance"", ""Gestational"", ""diabetes"", ""is...","[""O"", ""C"", ""C"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",93,93
...,...,...,...,...
284,"[""Clinical"", ""Preventive"", ""Service"", ""Recomme...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",29,29
285,"[""The"", ""AAFP"", ""also"", ""agrees"", ""that"", ""the...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",76,76
286,"[""While"", ""AAFP"", ""acknowledges"", ""that"", ""dia...","[""O"", ""O"", ""O"", ""O"", ""E"", ""O"", ""O"", ""O"", ""O"", ...",51,51
287,"[""The"", ""evidence"", ""did"", ""show"", ""a"", ""reduc...","[""O"", ""O"", ""O"", ""O"", ""O"", ""E"", ""E"", ""E"", ""E"", ...",35,35


In [ ]:
df_train_new.to_excel('/content/count_GD.xlsx')

In [ ]:
df_train_new = pd.read_excel('/content/count_GD.xlsx')

In [ ]:
df_train_new

,Unnamed: 0,sentences,label,token_count,label_count
0,0,"[""ImportanceGestational"", ""diabetes"", ""is"", ""d...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",93,93
1,1,"[""Rationale"", ""Pregnant"", ""persons"", ""at"", ""24...","[""O"", ""CO"", ""CO"", ""CO"", ""CO"", ""CO"", ""CO"", ""CO""...",32,32
2,2,"[""There"", ""is"", ""inadequate"", ""evidence"", ""tha...","[""O"", ""O"", ""S"", ""S"", ""O"", ""C"", ""C"", ""C"", ""C"", ...",20,20
3,3,"[""Benefits"", ""of"", ""early"", ""detection"", ""and""...","[""S"", ""O"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""O"", ...",62,62
4,4,"[""There"", ""is"", ""inadequate"", ""direct"", ""evide...","[""O"", ""O"", ""S"", ""S"", ""S"", ""O"", ""C"", ""C"", ""C"", ...",33,33
...,...,...,...,...,...
1048,1054,"[""The"", ""impact"", ""of"", ""the"", ""interventions""...","[""O"", ""C"", ""C"", ""C"", ""C"", ""O"", ""O"", ""O"", ""O"", ...",25,25
1049,1055,"[""Lifestyle"", ""interventions"", ""to"", ""prevent""...","[""C"", ""C"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",34,34
1050,1056,"[""Moreover"", ""the"", ""AAFP"", ""is"", ""concerned"",...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""S"", ""O"", ""C"", ...",32,32
1051,1057,"[""There"", ""is"", ""also"", ""concern"", ""that"", ""pe...","[""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ...",18,18


In [ ]:
for tokens in df_train_new['sentences']:
  print(json.loads(tokens))

['Type', '1', 'diabetes', '(', 'due', 'to', 'autoimmune', 'b-cell', 'destruction,', 'usually', 'leading', 'to', 'absolute', 'insulin', 'deficiency', ')2.']
['Type', '2', 'diabetes', '(', 'due', 'to', 'a', 'progressive', 'loss', 'of', 'adequate', 'b-cell', 'insulin', 'secretion', 'frequently', 'on', 'the', 'background', 'of', 'insulin', 'resistance', ')3.']
['Gestational', 'diabetes', 'mellitus', '(', 'diabetes', 'diagnosed', 'in', 'the', 'second', 'or', 'third', 'trimester', 'of', 'pregnancy', 'that', 'was', 'not', 'clearly', 'overt', 'diabetes', 'prior', 'to', 'gestation', ')4.']
['Specific', 'types', 'of', 'diabetes', 'due', 'to', 'other', 'causes,', 'e.g.,', 'monogenic', 'diabetes', 'syndromes', '(such', 'as', 'neonatal', 'diabetes', 'and', 'maturity-onset', 'diabetes', 'of', 'the', 'young),', 'diseases', 'of', 'the', 'exocrine', 'pancreas', '(such', 'as', 'cystic', 'fibrosis', 'and', 'pancreatitis),', 'and', 'drug-', 'or', 'chemical-induced', 'diabetes', '(such', 'as', 'with', 'glu

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'A', 1: 'E', 2: 'CO', 3: 'S', 4: 'C', 5: 'O'}


In [ ]:
print(X_raw[7])

['Occasionally,', 'patients', 'with', 'type', '2', 'diabetes', '<MD>may</MD>', 'present', 'with', 'DKA', '(6),', 'particularly', 'ethnic', 'minorities', '(7).']


In [ ]:
print(y_raw[7])

['O', 'C', 'C', 'C', 'C', 'C', 'O', 'E', 'E', 'E', 'O', 'O', 'CO', 'CO', 'O']


In [ ]:
RANDOM_SEED=42

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(X_raw, y_raw, test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_train_final = np.array(X_train_raw)
y_train_final = np.array(y_train_raw)

<ipython-input-19-100920d9c2b5>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_train_raw)
<ipython-input-19-100920d9c2b5>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_train_raw)


In [ ]:
print(np.shape(X_dev_raw))
print(np.shape(y_dev_raw))
print(np.shape(X_train_raw))

(58,)
(58,)
(231,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
X_train_raw

[['We',
  'suggest',
  'that',
  'all',
  'women',
  'with',
  'diabetes',
  'and',
  'preconceptional',
  'renal',
  'dysfunction',
  'have',
  'their',
  'renal',
  'function',
  'monitored',
  'regularly',
  'during',
  'pregnancy',
  '.'],
 ['SI',
  'conversion',
  'factor:',
  'To',
  'convert',
  'glucose',
  'values',
  'from',
  'mmol/L',
  'to',
  'mg/dL',
  'divide',
  'by',
  '0.0555',
  '.'],
 ['The',
  'Toronto',
  'Tri-',
  'Hospital',
  'Study',
  'showed',
  'continuous',
  'associations',
  'between',
  'maternal',
  'glycemia',
  'and',
  'adverse',
  'pregnancy',
  'outcomes',
  '(29).'],
 ['(1IQEEE)',
  '1.5',
  'c.',
  'We',
  'suggest',
  'that',
  'pregnant',
  'women',
  'with',
  'diabetes',
  'not',
  'known',
  'to',
  'have',
  'retinopathy',
  'have',
  'ocular',
  'assessment',
  'performed',
  'soon',
  'after',
  'conception',
  'and',
  'then',
  'periodically',
  'as',
  'indicated',
  'during',
  'pregnancy',
  '.'],
 ['Based',
  'on',
  'trial',
  'i

In [ ]:
# ensure our data is well-formed:
for ds in [X_train_raw, X_train_raw]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [test_texts, test_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# ensure our data is well-formed:
for ds in [train_texts, train_texts]:
  for row in ds:
    assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        # print(arr_offset)
        # print(type(arr_offset))

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
        #print(encoded_labels)
    return encoded_labels

In [ ]:
print(np.shape(train_labels))

(422,)


In [ ]:
print(train_texts[6])
print(train_labels[6])

['Correct', 'diagnosis', 'has', 'critical', 'implications', 'because', 'mostTable', '2.6-Most', 'common', 'causes', 'of', 'monogenic', 'diabetes', '(102)Gene', 'Inheritance', 'Clinical', 'featuresMODYNeonatal', 'diabetesGCK', 'AD', 'GCK-MODY', ':', 'stable,', 'nonprogressive', 'elevated', 'fasting', 'blood', 'glucose;', 'typically', 'does', 'notrequire', 'treatment;', 'microvascular', 'complications', 'are', 'rare;', 'small', 'rise', 'in', '2-h', 'PG', 'level', 'on', 'OGTT', '(,54', 'mg/dL', '[3', 'mmol/L])', 'HNF1A', 'AD', 'HNF1A-MODY', ':', 'progressive', 'insulin', 'secretory', 'defect', 'with', 'presentation', 'in', 'adolescence', 'or', 'earlyadulthood;', 'lowered', 'renal', 'threshold', 'for', 'glucosuria;', 'large', 'rise', 'in', '2-h', 'PG', 'level', 'on', 'OGTT', '(.90', 'mg/dL', '[5', 'mmol/L]);', 'sensitive', 'to', 'sulfonylureas', 'HNF4A', 'AD', 'HNF4A-MODY', ':', 'progressive', 'insulin', 'secretory', 'defect', 'with', 'presentation', 'in', 'adolescence', 'or', 'earlyadulth

In [ ]:
print(X_dev_raw[42])
print(y_dev_raw[42])

['For', 'fetal/neonatal', 'outcomes', ',', 'treatment', 'was', 'significantly', 'associated', 'with', 'lower', 'risk', 'of', 'shoulder', 'dystocia', '(4', 'trials;', '<DI>RR,', '0.42', '[95%', 'CI,', '0.23', 'to', '0.77];', 'I2', '=', '0%;', 'ARD,', '-1.3%', '[95%', 'CI,', '-4.3%', 'to-1.6%]</DI>),25,27,32,35', 'macrosomia', '(8', 'studies;', '<DI>RR,', '0.53', '[95%', 'CI,', '0.41', 'to0.68];', 'I2', '=', '42%;', 'ARD,', '-8.9%', '[-12.0%', 'to', '-5.9%]</DI>),25-27,29,31,32,35,36LGA', '(7', 'trials;', '<DI>RR,', '0.56', '[95%', 'CI,', '0.47', 'to', '0.66];', 'I2', '=', '0%;', 'ARD,', '-8.4%[95%', 'CI,', '-10.8%', 'to', '-6.1%]</DI>),26-28,31,32,35,36', 'and', 'NICU', 'admissions', '(5trials;', '<DI>RR,', '0.73', '[95%', 'CI,', '0.53', 'to', '0.99];', 'I2', '=', '0%;', 'ARD,', '-2.0%', '[95%CI,', '-4.5%', 'to', '0.5%]</DI>).26,28,31,32,36', 'Treatment', 'was', 'significantly', 'associated', 'with', 'reduced', 'risk', 'of', 'birth', 'injury', '(eg,', 'fracture', 'or', 'nerve', 'palsies

In [ ]:
print(test_texts[8])
print(test_labels[8])

['For', 'fetal/neonatal', 'outcomes', 'treatment', 'was', 'significantly', 'associated', 'with', 'lower', 'risk', 'of', 'shoulder', 'dystocia', '(4', 'trials', '<DI>RR', '042', '[95%', 'CI', '023', 'to', '077]', 'I2', '=', '0%', 'ARD', '13%', '[95%', 'CI', '43%', 'to16%]</DI>)25273235', 'macrosomia', '(8', 'studies', '<DI>RR', '053', '[95%', 'CI', '041', 'to068]', 'I2', '=', '42%', 'ARD', '89%', '[120%', 'to', '59%]</DI>)25272931323536LGA', '(7', 'trials', '<DI>RR', '056', '[95%', 'CI', '047', 'to', '066]', 'I2', '=', '0%', 'ARD', '84%[95%', 'CI', '108%', 'to', '61%]</DI>)262831323536', 'and', 'NICU', 'admissions', '(5trials', '<DI>RR', '073', '[95%', 'CI', '053', 'to', '099]', 'I2', '=', '0%', 'ARD', '20%', '[95%CI', '45%', 'to', '05%]</DI>)2628313236', 'Treatment', 'was', 'significantly', 'associated', 'with', 'reduced', 'risk', 'of', 'birth', 'injury', '(eg', 'fracture', 'or', 'nerve', 'palsies)', 'in', '3', 'trials', 'that', 'reported', 'at', 'least', '1', 'event', '(<DI>OR', '033'

In [ ]:
# encoded X
X_train = tokenize(train_texts)
X_dev = tokenize(X_dev_raw)
X_test= tokenize(test_texts)

y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
y_test = encode_tags(tags=test_labels, encodings=X_test, tag2id=tag2id)


for X in [X_train, X_dev, X_test]:
  X.pop("offset_mapping")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
df_train_new

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_dev, y_dev)
test_dataset = CustomDataset(X_test,y_test)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=y_train)

<ipython-input-28-2ab7c11a097d>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    report_to="wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()
run.finish()

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.794924
2,1.810000,1.777415
3,1.782300,1.749880
4,1.747700,1.714705
5,1.676100,1.674129
6,1.676100,1.634606
7,1.631200,1.583106
8,1.554900,1.528535
9,1.478500,1.459926
10,1.395000,1.383673


eval/loss,██▇▆▄▃▁▁▁▁▂▂▃▄▃▄▄▅▅▆▆▅▅▆▆▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆
eval/runtime,▄▆█▄▃▁▃▃▃▃▄▃▃▃▃▂▂▃▃▃▃▄▃▃▂▁▄▄▄▃▃▃▃▃▂▃▄▂▂▁
eval/samples_per_second,▅▃▁▅▆█▆▆▆▅▅▆▆▆▆▇▇▆▆▆▆▅▅▆▆█▅▅▅▆▆▆▅▆▇▅▅▇▇█
eval/steps_per_second,▅▃▁▅▆█▆▆▆▅▅▆▆▆▆▇▇▆▆▆▆▅▅▆▇█▅▅▅▆▆▆▅▆▇▅▅▇▇█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇███▇▇▆▆▆▅▅▄▄▃▃▂▂▁
train/loss,██▇▇▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
run.finish()

In [ ]:
######DistilBERT 18 epochs - validation loss is less


In [ ]:
!pip install -U tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.9 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.14.1 which is incompatible.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-09-29 23:04:18.724445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=18,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Currently logged in as: sgopala4. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.801213
2,1.793500,1.782652
3,1.764300,1.754170
4,1.734100,1.718604
5,1.673000,1.678244
6,1.673000,1.638773
7,1.622600,1.590270
8,1.548800,1.538713
9,1.480600,1.473094
10,1.377300,1.378840


In [ ]:
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
#indices = val_index
#final_pred = pd.Series(preds).tolist()

In [ ]:
preds

array([[5, 5, 5, ..., 2, 2, 2],
       [5, 5, 1, ..., 1, 1, 1],
       [5, 5, 5, ..., 0, 0, 0],
       ...,
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 0, 0],
       [0, 0, 0, ..., 4, 4, 4]])

In [ ]:
preds.shape

(115, 201)

In [ ]:
np.shape(y_test)

(115, 201)

In [ ]:
#flat_list = [item for sublist in preds for item in sublist]

In [ ]:
#np.shape(flat_list)

(202138,)

In [ ]:
#flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')


(23115,)
(23115,)
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'A', 1: 'E', 2: 'CO', 3: 'S', 4: 'C', 5: 'O'}


In [ ]:
targetname = ['A', 'E', 'CO','S', 'C', 'O']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)
#no determiners


       preds  gold_data
0          5       -100
1          5          5
2          5       -100
3          2          5
4          4       -100
...      ...        ...
23110      0       -100
23111      1       -100
23112      4       -100
23113      4       -100
23114      4       -100

[23115 rows x 2 columns]
       preds  gold_data
1          5          5
3          2          5
8          4          5
14         0          0
15         0          0
...      ...        ...
22929      5          4
22930      0          4
22936      4          4
22937      1          4
22940      5          5

[3879 rows x 2 columns]


In [ ]:
frame_combine_test_new

,preds,gold_data
1,2,5
3,2,5
8,2,5
14,2,3
15,2,3
...,...,...
22929,3,1
22930,4,1
22936,2,1
22937,2,1


In [ ]:
#distilbert base uncased
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           S       1.00      0.00      0.00       145
           E       0.49      0.01      0.02      2318
           O       0.45      0.47      0.46      3511
           C       1.00      0.00      0.00      1616
          CO       1.00      0.00      0.00      1394
           A       0.40      0.90      0.55      4146

    accuracy                           0.41     13130
   macro avg       0.72      0.23      0.17     13130
weighted avg       0.57      0.41      0.30     13130



In [ ]:
#distilbert base uncased - on final data
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.64      0.08      0.13       717
           C       0.07      0.03      0.04       411
           E       0.13      0.36      0.19       696
           A       0.15      0.12      0.14       838
           S       0.02      0.13      0.03        31
           O       0.34      0.23      0.27      1186

    accuracy                           0.18      3879
   macro avg       0.23      0.16      0.14      3879
weighted avg       0.29      0.18      0.18      3879



In [ ]:
#distilbert base uncased - 15 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.67      0.70      0.68      1186
           E       0.62      0.73      0.67       696
           C       0.61      0.44      0.51       411
           S       1.00      0.00      0.00        31
          CO       0.74      0.56      0.64       717
           A       0.66      0.76      0.70       838

    accuracy                           0.66      3879
   macro avg       0.71      0.53      0.53      3879
weighted avg       0.67      0.66      0.65      3879



In [ ]:
#distilbert base uncased - 18 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           A       0.66      0.74      0.69       838
           E       0.71      0.74      0.72       696
          CO       0.73      0.65      0.69       717
           S       1.00      0.00      0.00        31
           C       0.67      0.57      0.62       411
           O       0.70      0.73      0.72      1186

    accuracy                           0.69      3879
   macro avg       0.74      0.57      0.57      3879
weighted avg       0.70      0.69      0.69      3879



In [ ]:
###############BERT-base cased#########################################################

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification

In [ ]:
model_name = "bert-base-uncased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
import wandb

run = wandb.init()
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=20,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

run.finish()

wandb: Currently logged in as: sgopala4. Use `wandb login --relogin` to force relogin


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.876522
2,1.877700,1.845656
3,1.818800,1.799932
4,1.786500,1.743832
5,1.681600,1.680947
6,1.681600,1.617889
7,1.614800,1.536179
8,1.504000,1.446856
9,1.405900,1.354143
10,1.269200,1.249772


eval/loss,███▇▇▆▆▅▅▄▃▃▂▂▁▁▁▁▁▁
eval/runtime,▁▁▂▂▁▄▄▅▆▅█▆▆▃▄▃▅▆▆▄
eval/samples_per_second,██▇▇█▅▅▄▃▄▁▃▃▆▅▅▄▃▃▄
eval/steps_per_second,██▇▇█▅▅▄▃▄▁▃▃▆▅▅▄▃▃▄
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
train/learning_rate,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
train/loss,███▇▇▆▆▅▅▄▃▃▂▂▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
import wandb

run = wandb.init()
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

run.finish()

wandb: Currently logged in as: sgopala4. Use `wandb login --relogin` to force relogin


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.828396
2,1.794100,1.802506
3,1.774200,1.761552
4,1.723600,1.707994
5,1.631400,1.644879
6,1.631400,1.580963
7,1.563500,1.502385
8,1.479400,1.427888
9,1.393700,1.356160
10,1.275900,1.272593


eval/loss,██▆▅▃▂▁▁▂▁▂▃▃▄▄▅▅▅▅▆▅▇▆▇▇▅▆▇▆▆▆▆▆▆▆▆▆▆▆▆
eval/runtime,▁▁▃▄▆▇▇▆▇▇▇█▇▇▇▇▇▇▆▆▇▇▇▇▇▇█▆▆▇▇▆▇▇▇▇▆▇▇▆
eval/samples_per_second,██▅▄▃▁▂▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▃▂▂▂▃▂▂▂▂▂▂▂▃
eval/steps_per_second,██▅▄▃▁▂▃▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▃▂▂▂▃▂▂▂▂▂▂▂▃
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇███▇▇▆▆▆▅▅▄▄▃▃▂▂▁
train/loss,██▇▆▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
preds.shape

(115, 201)

In [ ]:
print(np.shape(y_test))

(115, 201)


In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(23115,)
(23115,)
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'CO', 1: 'A', 2: 'C', 3: 'O', 4: 'E', 5: 'S'}


In [ ]:
targetname = ['CO', 'A', 'C','O', 'E', 'S']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

       preds  gold_data
0          1       -100
1          3          3
2          2       -100
3          2          3
4          3       -100
...      ...        ...
23110      4       -100
23111      4       -100
23112      4       -100
23113      4       -100
23114      4       -100

[23115 rows x 2 columns]
       preds  gold_data
1          3          3
3          2          3
8          3          3
14         1          1
15         1          1
...      ...        ...
22929      1          2
22930      1          2
22936      1          2
22937      1          2
22940      3          3

[3879 rows x 2 columns]


In [ ]:
#bert base uncased
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           A       0.42      0.84      0.56      4146
           C       0.38      0.01      0.03      1616
          CO       0.27      0.00      0.00      1394
           O       0.50      0.56      0.52      3511
           S       1.00      0.00      0.00       145
           E       0.58      0.20      0.29      2318

    accuracy                           0.45     13130
   macro avg       0.52      0.27      0.23     13130
weighted avg       0.45      0.45      0.37     13130



In [ ]:
#bert base cased - on correct data
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.17      0.19      0.18       717
           C       0.12      0.03      0.05       411
           E       0.17      0.12      0.14       696
           A       0.24      0.21      0.23       838
           S       0.00      0.00      0.00        31
           O       0.31      0.42      0.36      1186

    accuracy                           0.24      3879
   macro avg       0.17      0.16      0.16      3879
weighted avg       0.22      0.24      0.22      3879



In [ ]:
#bert base cased - 15 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           S       1.00      0.00      0.00        31
           C       0.64      0.56      0.60       411
           E       0.68      0.79      0.73       696
           O       0.74      0.64      0.69      1186
           A       0.67      0.79      0.73       838
          CO       0.69      0.66      0.67       717

    accuracy                           0.69      3879
   macro avg       0.74      0.57      0.57      3879
weighted avg       0.70      0.69      0.69      3879



In [ ]:
#bert base uncased - 100 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.70      0.70      0.70       411
           E       0.77      0.81      0.79       696
          CO       0.74      0.68      0.71       717
           A       0.70      0.83      0.76       838
           S       0.59      0.52      0.55        31
           O       0.81      0.71      0.76      1186

    accuracy                           0.75      3879
   macro avg       0.72      0.71      0.71      3879
weighted avg       0.75      0.75      0.75      3879



In [ ]:
#bert base uncased - 20 epochs
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.71      0.68      0.70       717
           A       0.71      0.83      0.76       838
           C       0.69      0.59      0.63       411
           O       0.75      0.75      0.75      1186
           E       0.77      0.76      0.76       696
           S       1.00      0.00      0.00        31

    accuracy                           0.73      3879
   macro avg       0.77      0.60      0.60      3879
weighted avg       0.73      0.73      0.73      3879



In [ ]:
##############################################BioBERT-HuggingFace#################################################################

In [ ]:
from transformers import AutoModelForTokenClassification,RobertaTokenizerFast

In [ ]:
model_name = "fidukm34/biobert_v1.1_pubmed-finetuned-ner-finetuned-ner" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at fidukm34/biobert_v1.1_pubmed-finetuned-ner-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.812214
2,1.819700,1.760320
3,1.770300,1.690232


In [ ]:
preds

array([[5, 2, 5, ..., 3, 5, 5],
       [0, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 2, 2],
       [5, 5, 5, ..., 5, 5, 5]])

In [ ]:
preds.shape

(115, 215)

In [ ]:
np.shape(y_test)

(115, 215)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(24725,)
(24725,)
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'CO', 1: 'C', 2: 'E', 3: 'A', 4: 'S', 5: 'O'}


In [ ]:
targetname = ['CO', 'C', 'E','A', 'S', 'O']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

       preds  gold_data
0          5       -100
1          2          5
2          5       -100
3          0       -100
4          2       -100
...      ...        ...
24720      5       -100
24721      1       -100
24722      5       -100
24723      5       -100
24724      5       -100

[24725 rows x 2 columns]
       preds  gold_data
1          2          5
5          3          5
15         1          5
25         5          3
26         5          3
...      ...        ...
24525      5          1
24526      4          1
24532      1          1
24533      3          1
24536      5          5

[3879 rows x 2 columns]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           A       0.51      0.84      0.64      4375
           E       0.44      0.27      0.34      2312
           C       0.30      0.02      0.04      1732
          CO       0.39      0.16      0.23      1281
           S       1.00      0.00      0.00       169
           O       0.49      0.57      0.53      3795

    accuracy                           0.49     13664
   macro avg       0.52      0.31      0.29     13664
weighted avg       0.46      0.49      0.43     13664



In [ ]:
#BioBERT - on correct data
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.30      0.10      0.15       717
           C       0.14      0.11      0.13       411
           E       0.34      0.18      0.23       696
           A       0.28      0.09      0.14       838
           S       0.00      0.00      0.00        31
           O       0.37      0.81      0.50      1186

    accuracy                           0.33      3879
   macro avg       0.24      0.22      0.19      3879
weighted avg       0.30      0.33      0.27      3879



In [ ]:
######################BioBERT ------ dmis-lab/biobert-base-cased-v1.2

In [ ]:
model_name = "dmis-lab/biobert-base-cased-v1.2" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=100,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

run.finish()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.753870
2,1.772000,1.732488
3,1.734600,1.699386
4,1.697800,1.657935
5,1.599600,1.610136
6,1.599600,1.561829
7,1.540700,1.482121
8,1.437500,1.380280
9,1.315100,1.252946
10,1.122700,1.110236


wandb: ERROR Control-C detected -- Run data was not synced


In [ ]:
import wandb

run = wandb.init()

training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=16,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

run.finish()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.752795
2,1.780600,1.728591
3,1.745700,1.691962
4,1.689600,1.648997
5,1.609400,1.603402
6,1.609400,1.562940
7,1.543800,1.494435
8,1.449400,1.403159
9,1.337600,1.281316
10,1.150600,1.126793


eval/loss,███▇▇▇▆▆▅▄▂▂▁▁▁▁
eval/runtime,▇█▅▄▂▁▂▂▃▃▄▄▄▃▁▃
eval/samples_per_second,▂▁▄▅▇█▇▇▆▆▅▅▅▅█▆
eval/steps_per_second,▂▁▄▅▇█▇▇▆▆▅▅▅▆█▆
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇████
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇████
train/learning_rate,▁▂▂▃▄▄▅▅▆▇▇█
train/loss,███▇▇▆▆▅▄▂▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [ ]:
preds

array([[5, 5, 5, ..., 5, 5, 5],
       [5, 5, 1, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 0, ..., 5, 5, 5]])

In [ ]:
preds #second run

array([[5, 5, 5, ..., 5, 5, 5],
       [5, 5, 1, ..., 5, 5, 5],
       [5, 5, 5, ..., 5, 5, 5],
       ...,
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 5, 5, 5],
       [5, 5, 0, ..., 5, 5, 5]])

In [ ]:
preds.shape

(115, 220)

In [ ]:
np.shape(y_test)

(115, 220)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(25300,)
(25300,)
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'A', 1: 'E', 2: 'CO', 3: 'S', 4: 'C', 5: 'O'}


In [ ]:
targetname = ['A', 'E', 'CO','S', 'C', 'O']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

       preds  gold_data
0          5       -100
1          5          5
2          5       -100
3          5       -100
4          5          5
...      ...        ...
25295      5       -100
25296      5       -100
25297      5       -100
25298      5       -100
25299      5       -100

[25300 rows x 2 columns]
       preds  gold_data
1          5          5
4          5          5
9          5          5
15         0          0
16         0          0
...      ...        ...
25095      0          4
25096      0          4
25102      0          4
25103      0          4
25106      5          5

[3879 rows x 2 columns]


In [ ]:
#second run
targetname = ['S', 'CO', 'C','O', 'A', 'E']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

        preds  gold_data
0           3       -100
1           3          3
2           1       -100
3           4          3
4           3          3
...       ...        ...
191550      4       -100
191551      4       -100
191552      4       -100
191553      4       -100
191554      4       -100

[191555 rows x 2 columns]
        preds  gold_data
1           3          3
3           4          3
4           3          3
13          3          1
14          3          1
...       ...        ...
191125      4          4
191126      5          4
191127      4          4
191128      4          4
191129      4          4

[13664 rows x 2 columns]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       0.54      0.32      0.40      2312
           S       1.00      0.00      0.00       169
          CO       0.46      0.16      0.23      1281
           A       0.48      0.90      0.62      4375
           O       0.57      0.52      0.54      3795
           C       0.44      0.03      0.06      1732

    accuracy                           0.50     13664
   macro avg       0.58      0.32      0.31     13664
weighted avg       0.51      0.50      0.45     13664



In [ ]:
#second run
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           S       1.00      0.00      0.00       169
          CO       0.45      0.08      0.13      1281
           C       0.44      0.02      0.04      1732
           O       0.56      0.55      0.55      3795
           A       0.50      0.86      0.63      4375
           E       0.55      0.47      0.51      2312

    accuracy                           0.52     13664
   macro avg       0.58      0.33      0.31     13664
weighted avg       0.51      0.52      0.46     13664



In [ ]:
#######correct data######################
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.21      0.05      0.08       717
           C       0.08      0.01      0.02       411
           E       0.26      0.13      0.18       696
           A       0.24      0.40      0.30       838
           S       0.00      0.00      0.00        31
           O       0.29      0.46      0.36      1186

    accuracy                           0.26      3879
   macro avg       0.18      0.18      0.16      3879
weighted avg       0.24      0.26      0.22      3879



In [ ]:
#######correct data-8epochs######################
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           S       1.00      0.00      0.00        31
           A       0.56      0.29      0.38       838
           E       0.58      0.53      0.55       696
           C       0.46      0.01      0.03       411
           O       0.46      0.84      0.60      1186
          CO       0.51      0.46      0.48       717

    accuracy                           0.50      3879
   macro avg       0.60      0.36      0.34      3879
weighted avg       0.52      0.50      0.46      3879



In [ ]:
#######correct data-15epochs######################
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           O       0.73      0.76      0.75      1186
           S       1.00      0.00      0.00        31
           A       0.72      0.70      0.71       838
           E       0.84      0.71      0.77       696
          CO       0.67      0.80      0.73       717
           C       0.66      0.61      0.63       411

    accuracy                           0.72      3879
   macro avg       0.77      0.60      0.60      3879
weighted avg       0.73      0.72      0.72      3879



In [ ]:
#######correct data - 100 epochs######################
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           C       0.66      0.64      0.65       411
           E       0.86      0.75      0.80       696
          CO       0.71      0.65      0.68       717
           A       0.63      0.81      0.71       838
           S       0.62      0.52      0.56        31
           O       0.78      0.74      0.76      1186

    accuracy                           0.73      3879
   macro avg       0.71      0.68      0.69      3879
weighted avg       0.74      0.73      0.73      3879



In [ ]:
##############correct data - 16 epochs###################
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           A       0.65      0.85      0.73       838
           E       0.82      0.75      0.78       696
          CO       0.80      0.63      0.71       717
           S       1.00      0.00      0.00        31
           C       0.62      0.69      0.65       411
           O       0.78      0.74      0.76      1186

    accuracy                           0.73      3879
   macro avg       0.78      0.61      0.61      3879
weighted avg       0.75      0.73      0.73      3879



In [ ]:
print(df1.shape)
print(frame_combine_test_new.shape)

(13664, 2)
(13664, 2)


In [ ]:
frame_combine_test_new.to_excel('/content/preds_gold.xlsx')

In [ ]:
flat_list1 = [item1 for sublist1 in train_labels for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in train_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']

(13079,)
       index          texts
0          0            The
1          1       benefits
2          2             of
3          3  breastfeeding
4          4        greatly
...      ...            ...
13074     19             by
13075     20            the
13076     21           most
13077     22         common
13078     23       variants

[13079 rows x 2 columns]


In [ ]:
gold = pd.DataFrame(flat_list1)

In [ ]:
train_data = pd.concat([df1,gold],axis=1)

In [ ]:
train_data

,index,texts,0
0,0,The,O
1,1,benefits,S
2,2,of,O
3,3,breastfeeding,C
4,4,greatly,E
...,...,...,...
13074,19,by,O
13075,20,the,O
13076,21,most,O
13077,22,common,O


In [ ]:
train_data.to_excel('/content/train_GD.xlsx')

In [ ]:
###############################################BioBERT --------dmis-lab/TinyPubMedBERT-v1.0

In [ ]:
model_name = "dmis-lab/TinyPubMedBERT-v1.0" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/TinyPubMedBERT-v1.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.791182
2,1.801100,1.770738
3,1.765200,1.740597


In [ ]:
preds

array([[5, 1, 3, ..., 0, 0, 3],
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 5, 0],
       ...,
       [5, 3, 5, ..., 0, 0, 0],
       [5, 5, 5, ..., 0, 0, 0],
       [5, 5, 3, ..., 0, 0, 0]])

In [ ]:
preds.shape

(115, 196)

In [ ]:
np.shape(y_test)

(115, 196)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

(22540,)
(22540,)
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]
      index                    texts
0         0                   CYSTIC
1         1         FIBROSIS-RELATED
2         2  DIABETESRecommendations
3         3                   Annual
4         4                screening
...     ...                      ...
3874     13                       of
3875     14      posttransplantation
3876     15                 diabetes
3877     16                 mellitus
3878     17                        .

[3879 rows x 2 columns]


In [ ]:
print(id2tag)

{0: 'CO', 1: 'C', 2: 'E', 3: 'A', 4: 'S', 5: 'O'}


In [ ]:
targetname = ['CO', 'C', 'E','A', 'S', 'O']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

       preds  gold_data
0          5       -100
1          1          5
2          3          5
3          5       -100
4          3       -100
...      ...        ...
22535      0       -100
22536      0       -100
22537      0       -100
22538      0       -100
22539      0       -100

[22540 rows x 2 columns]
       preds  gold_data
1          1          5
2          3          5
5          5          5
11         3          3
12         3          3
...      ...        ...
22359      5          1
22360      5          1
22362      5          1
22363      3          1
22364      2          5

[3879 rows x 2 columns]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

           E       1.00      0.00      0.00      2312
           S       1.00      0.00      0.00       169
          CO       1.00      0.00      0.00      1281
           A       0.32      1.00      0.49      4375
           O       0.50      0.00      0.01      3795
           C       1.00      0.00      0.00      1732

    accuracy                           0.32     13664
   macro avg       0.80      0.17      0.08     13664
weighted avg       0.64      0.32      0.16     13664



In [ ]:
#####correct data###########
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

              precision    recall  f1-score   support

          CO       0.22      0.01      0.02       717
           C       0.02      0.00      0.00       411
           E       0.00      0.00      0.00       696
           A       0.25      0.33      0.28       838
           S       0.00      0.00      0.00        31
           O       0.33      0.74      0.46      1186

    accuracy                           0.30      3879
   macro avg       0.14      0.18      0.13      3879
weighted avg       0.20      0.30      0.20      3879



In [ ]:
################################################

In [ ]:
model_name = "dmis-lab/bern2-ner" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name,add_prefix_space=True)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at dmis-lab/bern2-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([6, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([6]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: ignored

In [ ]:
preds

In [ ]:
preds.shape

In [ ]:
np.shape(y_test)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

In [ ]:
print(id2tag)

In [ ]:
targetname = ['E', 'S', 'CO','A', 'O', 'C']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

In [ ]:
##############################################BioBERT-large---------------- dmis-lab/biobert-large-cased-v1.1

In [ ]:
model_name = "dmis-lab/biobert-large-cased-v1.1" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/BioBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags),ignore_mismatched_sizes=True)

trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
)

trainer.train()
pred = trainer.predict(test_dataset)
preds = pred.predictions.argmax(-1)
# indices = val_index
#final_pred = pd.Series(preds).tolist()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-large-cased-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: ignored

In [ ]:
preds

In [ ]:
preds.shape

In [ ]:
np.shape(y_test)

In [ ]:
flat_list = [item for sublist in preds for item in sublist]
print(np.shape(flat_list))
flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
print(np.shape(flat_list1))
flat_list_testtexts = [(ind,item1) for sublist1 in test_texts for ind,item1 in enumerate(sublist1)]
flat_list_testtexts_frame = pd.DataFrame(flat_list_testtexts)
flat_list_testtexts_frame.columns = ['index','texts']
print(flat_list_testtexts_frame)
df1 = flat_list_testtexts_frame[flat_list_testtexts_frame.texts != ' ']
print(df1)
df1.to_excel('/content/text_train.xlsx')

In [ ]:
print(id2tag)

In [ ]:
targetname = ['E', 'S', 'CO','A', 'O', 'C']
lis_preds_frame = pd.DataFrame(flat_list)
lis_test_labels_frame = pd.DataFrame(flat_list1)
frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
frame_combine_test.columns = ['preds','gold_data']
print(frame_combine_test)

frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
print(frame_combine_test_new)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()

In [ ]:
################################################BioBERT#############################################################################

In [ ]:
!git clone https://github.com/dmis-lab/biobert.git

Cloning into 'biobert'...
remote: Enumerating objects: 359, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 359 (delta 46), reused 51 (delta 23), pack-reused 278
Receiving objects: 100% (359/359), 516.14 KiB | 2.04 MiB/s, done.
Resolving deltas: 100% (208/208), done.


In [ ]:
cd biobert

/content/biobert


In [ ]:
#!pip install -r requirements.txt

In [ ]:
!./download.sh

BIOBERT_DATA not set; downloading to default path ('data').
--2023-09-15 19:12:13--  http://nlp.dmis.korea.edu/projects/biobert-2020-checkpoints/datasets.tar.gz
Resolving nlp.dmis.korea.edu (nlp.dmis.korea.edu)... 163.152.163.168
Connecting to nlp.dmis.korea.edu (nlp.dmis.korea.edu)|163.152.163.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29610233 (28M) [application/x-gzip]
Saving to: ‘./data.tar.gz’

./data.tar.gz       100%[===================>]  28.24M  1.56MB/s    in 27s     

2023-09-15 19:12:42 (1.05 MB/s) - ‘./data.tar.gz’ saved [29610233/29610233]

datasets/
datasets/RE/
datasets/RE/GAD/
datasets/RE/GAD/6/
datasets/RE/GAD/6/test.tsv
datasets/RE/GAD/6/dev.tsv
datasets/RE/GAD/6/train.tsv
datasets/RE/GAD/7/
datasets/RE/GAD/7/test.tsv
datasets/RE/GAD/7/dev.tsv
datasets/RE/GAD/7/train.tsv
datasets/RE/GAD/5/
datasets/RE/GAD/5/test.tsv
datasets/RE/GAD/5/dev.tsv
datasets/RE/GAD/5/train.tsv
datasets/RE/GAD/8/
datasets/RE/GAD/8/test.tsv
datasets/RE/GAD/8/d

In [ ]:
pwd

'/content/biobert'

In [ ]:
!tar -xzf '/content/biobert_v1.1_pubmed.tar.gz' -C models

In [ ]:
# train_tex = pd.DataFrame(train_texts)
# train_lab = pd.DataFrame(train_labels)
# dev_tex = pd.DataFrame(X_dev_raw)
# dev_lab = pd.DataFrame(y_dev_raw)
# test_tex = pd.DataFrame(test_texts)
# test_lab = pd.DataFrame(test_labels)

In [ ]:
# train = pd.concat([train_tex,train_lab],axis=1)
# dev = pd.concat([dev_tex,dev_lab],axis=1)
# test = pd.concat([test_tex,test_lab],axis=1)

In [ ]:
RANDOM_SEED=42

In [ ]:
# Our data is split into sentences


# FIXME: we also need test data!
X_train_raw, X_dev_raw, y_train_raw, y_dev_raw = train_test_split(df_train_new['sentences'], df_train_new['label'], test_size=.2, random_state=RANDOM_SEED)

In [ ]:
X_train_final = np.array(X_train_raw)
y_train_final = np.array(y_train_raw)

In [ ]:
import os
k=2
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
metrics=[]
i=0
for train_index, val_index in kf.split(X_train_final,y_train_final):
  train_texts, test_texts =X_train_final[train_index].tolist(),X_train_final[val_index].tolist()
  train_labels, test_labels = y_train_final[train_index].tolist(),y_train_final[val_index].tolist()

In [ ]:
train_tex = pd.DataFrame(train_texts)
train_lab = pd.DataFrame(train_labels)
dev_tex = pd.DataFrame(X_dev_raw)
dev_lab = pd.DataFrame(y_dev_raw)
test_tex = pd.DataFrame(test_texts)
test_lab = pd.DataFrame(test_labels)

In [ ]:
train = pd.concat([train_tex,train_lab],axis=1)
dev = pd.concat([dev_tex,dev_lab],axis=1)
test = pd.concat([test_tex,test_lab],axis=1)

In [ ]:
train.columns = ['text', 'labels']

In [ ]:
dev.columns = ['text', 'labels']
test.columns = ['text', 'labels']

In [ ]:
train

,text,labels
0,"[""The"", ""benefits"", ""of"", ""breastfeeding"", ""gr...","[""O"", ""S"", ""O"", ""C"", ""E"", ""E"", ""E"", ""E"", ""E"", ..."
1,"[""It"", ""emphasizes"", ""healthy"", ""food"", ""choic...","[""O"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ..."
2,"[""Consideration"", ""was"", ""given"", ""to"", ""gluco...","[""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ..."
3,"[""There"", ""fore"", ""the"", ""Task"", ""Force"", ""rec...","[""O"", ""O"", ""C"", ""C"", ""C"", ""A"", ""A"", ""A"", ""A"", ..."
4,"[""The"", ""USPSTF"", ""examined"", ""whether"", ""meas...","[""C"", ""C"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ""A"", ..."
...,...,...
416,"[""Although"", ""not"", ""shown"", ""to"", ""be"", ""dele...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ..."
417,"[""Other"", ""Conditions"", ""Altering"", ""the"", ""Re...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ..."
418,"[""Once"", ""hyperglycemia"", ""occurs"", ""patients""...","[""O"", ""C"", ""O"", ""CO"", ""CO"", ""CO"", ""CO"", ""CO"", ..."
419,"[""1IQEEE"", ""15"", ""c"", ""We"", ""suggest"", ""that"",...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""CO"", ""CO"", ""CO..."


In [ ]:
#tag2id = {'A' : 0,'C' : 1, 'CO' : 2, 'O': 3, 'S': 4,'E': 5}

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_new = train[["text", "labels"]].rename(columns={"text": "X", "labels": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_dev_new = dev[["text", "labels"]].rename(columns={"text": "X", "labels": "y"})
# load each row's data as a list of strings
X_dev_raw, y_dev_raw = [json.loads(tokens) for tokens in dataset_dev_new.X.values], [json.loads(labels) for labels in dataset_dev_new.y.values]

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test_new = test[["text", "labels"]].rename(columns={"text": "X", "labels": "y"})
# load each row's data as a list of strings
X_test_raw, y_test_raw = [json.loads(tokens) for tokens in dataset_test_new.X.values], [json.loads(labels) for labels in dataset_test_new.y.values]

In [ ]:
# unique_tags = set(label for labels in y_raw for label in labels)
# tag2id = {tag: id for id, tag in enumerate(unique_tags)}
# id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
# unique_tags

{'A', 'C', 'CO', 'E', 'O', 'S'}

In [ ]:
# labels = [[tag2id[tag] for tag in doc] for doc in y_raw]

In [ ]:
# labels_dev = [[tag2id[tag] for tag in doc] for doc in y_dev_raw]
# labels_test = [[tag2id[tag] for tag in doc] for doc in y_test_raw]

In [ ]:
#type(labels)

In [ ]:
# labels

[[3, 3, 3, 3, 4, 3, 1, 1, 1, 1, 1, 1, 1, 3],
 [3,
  3,
  3,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  3,
  3,
  3,
  3,
  3],
 [3,
  3,
  3,
  3,
  3,
  3,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 [5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  0,
  0,
  0,
  0,
  0],
 [3, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [3, 3, 2, 2, 2],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  3],
 [3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 2, 3, 3, 3, 4, 4, 3, 1],
 [3, 3, 3, 3, 3

In [ ]:
# X_raw

[['Those',
  'trials',
  'found',
  'modest',
  'benefits',
  'including',
  'reduced',
  'rates',
  'of',
  'largeforgestationalage',
  'births',
  'and',
  'preeclampsia',
  '145146'],
 ['Data',
  'Sources',
  'and',
  'Searches',
  'Ovid',
  'MEDLINE',
  'and',
  'EMBASE',
  'and',
  'CINAHL',
  'via',
  'EBSCOhost',
  'were',
  'searched',
  'from',
  '2010',
  'to',
  'May',
  '22',
  '2020',
  'eMethods',
  '1',
  'in',
  'the',
  'Supple',
  'ment'],
 ['ResultsAcross',
  'the',
  'KQs',
  'reported',
  'in',
  'this',
  'manuscript',
  'a',
  'total',
  'of',
  '76',
  'studies',
  'were',
  'included',
  '18',
  'RCTs',
  'different',
  'screening',
  'strategies',
  'n',
  '27',
  '1961824',
  'treatment',
  'benefits',
  'and',
  'harms',
  'n',
  '40452535',
  '2',
  'nonrandomized',
  'controlled',
  'intervention',
  'studies',
  'of',
  'treatment',
  'n',
  '1903637',
  'and',
  '56',
  'observational',
  'studies',
  'screening',
  'benefits',
  'n',
  '43363841',
  'sc

In [ ]:
flat_list_train_texts = [item for sublist in X_raw for item in sublist]
flat_list_train_label = [item for sublist in y_raw for item in sublist]

In [ ]:
train = pd.DataFrame(zip(flat_list_train_texts,flat_list_train_label))

In [ ]:
flat_list_dev_texts = [item for sublist in X_dev_raw for item in sublist]
flat_list_dev_label = [item for sublist in y_dev_raw for item in sublist]

In [ ]:
dev = pd.DataFrame(zip(flat_list_dev_texts,flat_list_dev_label))

In [ ]:
flat_list_test_texts = [item for sublist in X_test_raw for item in sublist]
flat_list_test_label = [item for sublist in y_test_raw for item in sublist]

In [ ]:
test = pd.DataFrame(zip(flat_list_test_texts,flat_list_test_label))

In [ ]:
# train['tokens'] = pd.Series(X_raw)
# train['labels_final'] = pd.Series(y_raw)
# train['ner_tags'] = pd.Series(labels)

In [ ]:
# dev['tokens'] = pd.Series(X_dev_raw)
# dev['labels_final'] = pd.Series(y_dev_raw)
# dev['ner_tags'] = pd.Series(labels_dev)

In [ ]:
# test['tokens'] = pd.Series(X_test_raw)
# test['labels_final'] = pd.Series(y_test_raw)
# test['ner_tags'] = pd.Series(labels_dev)

In [ ]:
# train_final = pd.concat([train['tokens'],train['ner_tags']],axis=1)
# dev_final = pd.concat([dev['tokens'],dev['ner_tags']],axis=1)
# test_final = pd.concat([test['tokens'],test['ner_tags']],axis=1)

In [ ]:
# train_final

,tokens,ner_tags
0,"[Those, trials, found, modest, benefits, inclu...","[3, 3, 3, 3, 4, 3, 1, 1, 1, 1, 1, 1, 1, 3]"
1,"[Data, Sources, and, Searches, Ovid, MEDLINE, ...","[3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[ResultsAcross, the, KQs, reported, in, this, ...","[3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, ..."
3,"[If, the, plasma, glucose, level, measured, 1,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
4,"[a, Risk, factors, included, BMI, 30, or, grea...","[3, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...
417,"[Evidence, Pregnancy, outcomes, in, women, exp...","[4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, ..."
418,"[Other, Conditions, Altering, the, Relationshi...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, ..."
419,"[Once, hyperglycemia, occurs, patients, with, ...","[3, 2, 3, 5, 5, 5, 5, 5, 5, 3, 3, 4, 3, 1, 1, ..."
420,"[If, the, degree, of, retinopathy, warrants, t...","[5, 5, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train.to_csv('/content/data/train.tsv', sep="\t",header=None)
dev.to_csv('/content/data/train_dev.tsv',sep="\t",header=None)
test.to_csv('/content/data/test.tsv',sep='\t',header=None)

In [ ]:
# X_train = tokenize(train_texts)
# X_dev = tokenize(X_dev_raw)
# X_test= tokenize(test_texts)

# y_train = encode_tags(tags=train_labels, encodings=X_train, tag2id=tag2id)
# y_dev = encode_tags(tags=y_dev_raw, encodings=X_dev, tag2id=tag2id)
# y_test = encode_tags(tags=test_labels, encodings=X_test, tag2id=tag2id)

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.7
from IPython.display import clear_output
clear_output()
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1

# Choose one of the given alternatives:
!sudo update-alternatives --config python3

# This one used to work but now NOT(for me)!
# !sudo update-alternatives --config python

# Check the result
!python3 --version

# Attention: Install pip (... needed!)
!sudo apt install python3-pip

update-alternatives: using /usr/bin/python3.7 to provide /usr/bin/python3 (python3) in auto mode
There is only one alternative in link group python3 (providing /usr/bin/python3): /usr/bin/python3.7
Nothing to configure.
Python 3.7.17
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python3-pip python3-setuptools python3-wheel
0 upgraded, 3 newly installed, 0 to remove and 27 not upgraded.
Need to get 1,677 kB of archives.
After this operation, 8,965 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-setuptools all 59.6.0-1.2ubuntu0.22.04.1 [339 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-wheel all 0.37.1-2ubuntu0.22.04.1 [32.0 kB]
Get:3 http://archive.ubuntu.com/ub

In [ ]:
!sudo apt-get install --reinstall python3.7-distutils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.7-lib2to3
The following NEW packages will be installed:
  python3.7-distutils python3.7-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,229 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-lib2to3 all 3.7.17-1+jammy1 [124 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-distutils all 3.7.17-1+jammy1 [189 kB]
Fetched 313 kB in 2s (162 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize fronte

In [ ]:
!python3.7 -m pip install tensorflow-gpu==1.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.4/503.4 KB 49.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.5 MB/s eta 0:00

In [ ]:
!pip install protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.3
    Uninstalling protobuf-4.24.3:
      Successfully uninstalled protobuf-4.24.3


In [ ]:
#python run_re.py --task_name=$TASK_NAME --do_train=true --do_eval=true --do_predict=true --vocab_file=$BIOBERT_DIR/vocab.txt --bert_config_file=$BIOBERT_DIR/bert_config.json --init_checkpoint=$BIOBERT_DIR/model.ckpt-1000000 --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --do_lower_case=false --data_dir=$RE_DIR --output_dir=$OUTPUT_DIR

In [ ]:
!python run_ner.py --do_train=true --do_eval=true --vocab_file='/content/biobert/models/biobert_v1.1_pubmed/vocab.txt' --bert_config_file='/content/biobert/models/biobert_v1.1_pubmed/bert_config.json' --init_checkpoint='/content/biobert/models/biobert_v1.1_pubmed/model.ckpt-1000000' --num_train_epochs=10.0 --data_dir='/content/data' --output_dir='/content/output'


run_ner.py:648: UserWarning: Flag --task_name has a non-None default value; therefore, mark_flag_as_required will pass even if flag is not specified in the command line!
  flags.mark_flag_as_required("task_name")


W0915 19:27:20.056978 131996184875520 module_wrapper.py:139] From run_ner.py:467: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0915 19:27:20.057172 131996184875520 module_wrapper.py:139] From run_ner.py:467: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0915 19:27:20.057363 131996184875520 module_wrapper.py:139] From /content/biobert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0915 19:27:20.057905 131996184875520 module_wrapper.py:139] From run_ner.py:482: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more inf

In [ ]:
################################attention-BiLSTM#######################################################

In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


In [ ]:
#X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

In [ ]:
#n_unique_words = 10000

In [ ]:
# maxlen = 200
# x_train = sequence.pad_sequences(X_train_raw, maxlen=maxlen)
# x_test = sequence.pad_sequences(X_dev_raw, maxlen=maxlen)
# y_train = np.array(y_train_raw)
# y_test = np.array(y_dev_raw)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'GD',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_dataset = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_dataset = tf.keras.utils.text_dataset_from_directory(
    'GD',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: ignored

In [ ]:
import numpy as np
import pandas as pd

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [ ]:

VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(X_train_raw.map(lambda text, label: text))

AttributeError: ignored

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
dtype = torch.FloatTensor

# Bi-LSTM(Attention) Parameters
embedding_dim = 2
n_hidden = 5 # number of hidden units in one cell
num_classes = 2  # 0 or 1


In [ ]:
import pandas as pd
data_GD = pd.read_excel('/content/count_GD.xlsx')